### Evaluation TechnoVerde S.A.

Podemos evaluar diferentes partes de nuestro sistema:

- Respuestas del agente
- Llamado a herramientas
- Utilidad del RAG
- Etc.

Y podemos utilizar diferentes tipos de evaluadores:

- LLM as a judge evaluator
- Programmatic evaluator

In [ ]:
from langsmith import Client
from langsmith.evaluation import evaluate
from langsmith.schemas import Example, Run

# Initialize LangSmith client
client = Client()

![Texto alternativo](https://docs.smith.langchain.com/assets/images/langsmith_summary-7d48643f3e4c50d77234e13feb95144d.png)

### 1. Evaluando respuestas del agente y la utilidad del RAG (LLM as a judge)

#### Dataset 

In [ ]:
dataset = client.create_dataset(
    "Atencion al cliente",
    description="Un dataset para evaluar las respuestas del agente en temas de atencion al cliente"
)

In [ ]:
#Data sintetica creada con Claude 3.5 Sonnet

client.create_examples(
    inputs=[
        {"input": "¿Cuáles son los canales de atención al cliente que ofrece TechnoVerde?"},
        {"input": "¿Cuál es el tiempo de respuesta objetivo para los correos electrónicos?"},
        {"input": "¿Cuánto tiempo tengo para devolver un producto?"},
        {"input": "¿Qué cubre la garantía de los productos TechnoVerde?"},
        {"input": "¿Cómo funciona el programa de lealtad de TechnoVerde?"},
        {"input": "¿Qué medidas toma TechnoVerde para proteger los datos de los clientes?"},
        {"input": "¿Qué tipo de formación reciben los empleados de atención al cliente?"},
        {"input": "¿Cómo maneja TechnoVerde las situaciones de crisis?"},
        {"input": "¿Qué tecnologías innovadoras utiliza TechnoVerde en su atención al cliente?"},
        {"input": "¿Cómo asegura TechnoVerde la accesibilidad en su atención al cliente?"}
    ],
    outputs=[
        {"response": "TechnoVerde S.A. ofrece múltiples canales de atención al cliente: una línea telefónica gratuita (0800-TECNO-VERDE) disponible 24/7, correo electrónico (soporte@technoverde.com), chat en vivo en su sitio web (www.technoverde.com), redes sociales (@TechnoVerdeOficial en Facebook, Twitter e Instagram), y atención presencial en sus tiendas de lunes a sábado de 9:00 a 20:00."},
        {"response": "Según la política de atención al cliente de TechnoVerde, el tiempo de respuesta objetivo para los correos electrónicos es de menos de 4 horas hábiles."},
        {"response": "De acuerdo con la política de reembolsos y devoluciones de TechnoVerde, los clientes pueden devolver productos no utilizados dentro de los 30 días posteriores a la compra para obtener un reembolso completo o un cambio."},
        {"response": "Todos los productos TechnoVerde están cubiertos por una garantía mínima de 2 años. Algunos productos premium tienen garantías extendidas de hasta 5 años. Además, la empresa ofrece soporte técnico gratuito durante toda la vida útil del producto."},
        {"response": "El programa de lealtad de TechnoVerde funciona de la siguiente manera: los clientes son automáticamente inscritos al realizar su primera compra, ganan 1 punto por cada $10 gastados, y hay cuatro niveles de membresía (Verde, Plata, Oro y Platino) con beneficios crecientes. Los beneficios incluyen acceso anticipado a nuevos productos, invitaciones a eventos exclusivos y descuentos especiales."},
        {"response": "TechnoVerde toma varias medidas para proteger los datos de los clientes. Solo recopilan los datos necesarios para proporcionar sus servicios y mejorar la experiencia del cliente. Todos los datos se almacenan de forma segura en servidores encriptados y se implementan medidas de seguridad de vanguardia para proteger la información contra accesos no autorizados."},
        {"response": "Los empleados de atención al cliente de TechnoVerde reciben un programa de formación inicial intensivo de 4 semanas que cubre conocimiento de productos y servicios, habilidades de comunicación y resolución de problemas, políticas y procedimientos de la empresa, y uso de sistemas de atención al cliente. Además, se ofrecen programas de formación continua mensuales para mantenerlos actualizados."},
        {"response": "TechnoVerde mantiene un plan detallado de continuidad del negocio para garantizar la atención al cliente en casos de crisis como desastres naturales, fallos tecnológicos o pandemias. En situaciones de crisis, se comprometen a informar proactivamente a los clientes sobre cualquier interrupción del servicio, proporcionar actualizaciones regulares y ofrecer soluciones alternativas cuando sea posible."},
        {"response": "TechnoVerde utiliza varias tecnologías innovadoras en su atención al cliente, incluyendo soluciones de Inteligencia Artificial y chatbots para proporcionar respuestas instantáneas y analizar el sentimiento del cliente. También implementan tecnología de Realidad Aumentada (RA) para guiar a los clientes en la instalación y resolución de problemas de productos, así como para proporcionar demostraciones virtuales."},
        {"response": "TechnoVerde asegura la accesibilidad en su atención al cliente mediante un enfoque de diseño universal. Esto incluye un sitio web y aplicaciones móviles compatibles con lectores de pantalla, opciones de comunicación para clientes con discapacidad auditiva o del habla, y materiales de soporte en múltiples idiomas y formatos. Además, todo el personal de atención al cliente recibe formación regular sobre diversidad e inclusión."}
    ],
    dataset_id=dataset.id
)


#### App

In [ ]:
#AI App

from agent import Agent
from langchain_chroma import Chroma
from tools import create_retriever_tool_from_vectorstore, create_get_client_info_tool
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
from langchain_openai import OpenAIEmbeddings
persist_directory = "./chroma_db"
vectorstore = Chroma(
    collection_name="rag-chroma",
    embedding_function=OpenAIEmbeddings(),
    persist_directory=persist_directory
)

tools = [create_retriever_tool_from_vectorstore(vectorstore), create_get_client_info_tool()]

from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
        ("system", "Sos un asistente que responde preguntas sobre la empresa TechnoVerde S.A. Para preguntas relacionadas a la empresa, responde utilizando la informacion que tenes disponible sobre la misma, no inventes informacion. Si no conoces la respuesta, simplemente decí que no lo sabes y disculpate por no poder ayudar"),
    ])

In [ ]:
def start_conversation(input_dict):
    try:
        agent = Agent(model_type="openai", prompt=template, tools=tools)
        human_message = HumanMessage(content=input_dict['input'])
        state = agent.invoke([human_message])
        
        if isinstance(state, dict) and "messages" in state and len(state["messages"]) > 0:
            messages = state["messages"]
            result = {"answer": "", "retrieved_content": ""}
            
            for message in messages:
                if isinstance(message, ToolMessage) and message.name == "retrieve_company_docs":
                    result["retrieved_content"] = message.content
            
            last_message = messages[-1]
            if isinstance(last_message, AIMessage):
                result["answer"] = last_message.content
            
            return result
        
        return {"answer": "No AI response found in the state.", "retrieved_content": ""}
    except Exception as e:
        return {"error": str(e), "retrieved_content": ""}

In [ ]:
start_conversation({"input": "Quien es el CEO de TechnoVerde?"})


#### Evaluators

In [ ]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

answer_correctness_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "answer_correctness": """Is the Assistant's Answer aligned with the Ground Truth Answer? A score of [[1]] means that the
            Assistant answer contains is not at all based upon / grounded in the Groun Truth Answer. A score of [[5]] means 
            that the Assistant answer contains some information (e.g., a hallucination) that is not captured in the Ground Truth 
            Answer. A score of [[10]] means that the Assistant answer is fully based upon the in the Ground Truth Answer."""
        },
        # If you want the score to be saved on a scale from 0 to 1
        "normalize_by": 10,
        "key": "answer_correctness"
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": example.outputs["response"],
        "input": example.inputs["input"],
    },
)

dataset_name = "Atencion al cliente"
experiment_results = evaluate(
    start_conversation,
    data=dataset_name,
    evaluators=[answer_correctness_evaluator],
    experiment_prefix="answer-correctness-gpt-4o",

)


In [ ]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

# Evaluator for Document Relevance
document_relevance_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "document_relevance": """Evaluates the relevance of retrieved articles in relation to the user query. 
            A score of [[1]] indicates that the retrieved articles do not make any sense in the context of the user query. 
            A score of [[5]] means the retrieved articles are somewhat relevant but miss key elements related to the query. 
            A score of [[10]] shows that the retrieved articles are highly relevant and fully address the user query."""
        },
        "normalize_by": 10,
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["retrieved_content"],  # Retrieved context from your function
        "reference": example.outputs["response"],  # Ground truth response
        "input": example.inputs["input"],
    },
)

dataset_name = "Atencion al cliente"
experiment_results = evaluate(
    start_conversation,
    data=dataset_name,
    evaluators=[document_relevance_evaluator],
    experiment_prefix="RAG-document-relevance-gpt-4o",

)


### 2. Evaluando el llamado a la tool de informacion del cliente (Programmatic evaluator)

#### Dataset

In [ ]:
dataset = client.create_dataset(
    "Llamado a la tool de informacion del cliente",
    description="Un dataset para evaluar el lamado a la tool de informacion del cliente con los args correctos"
)

In [ ]:
#Data sintetica creada con Claude 3.5 Sonnet - Corregir los outputs para que sean los correctos

client.create_examples(
    inputs=[
        {"input": "Hola, necesito consultar mi información de cuenta. Mi número de cliente es TV001. ¿Me podrías ayudar?"},
        {"input": "Buenas, creo que mi ID es algo así como TV cero cero dos, ¿podrías verificar mi cuenta?"},
        {"input": "Hola, soy Ana Martínez y quisiera saber mi puntuación de sostenibilidad. No recuerdo mi ID de cliente."},
        {"input": "Necesito ayuda con mi cuenta, mi ID es TV004."},
        {"input": "Hola, instalé paneles solares y un termostato inteligente con ustedes. ¿Podrían decirme mi saldo y puntuación de sostenibilidad? Creo que mi ID es TV001."},
        {"input": "¿Qué tal? Quiero checar mi cuenta pero no recuerdo mi número de cliente. Soy Carlos de Green Business."},
        {"input": "Hola, mi ID de cliente es TV cero cero... creo que tres. Necesito mi información de cuenta."},
        {"input": "Necesito asistencia, mi número de cliente es TB002."},
        {"input": "Hola, soy María González. Instalé paneles solares y un termostato inteligente el año pasado. Quisiera saber mi saldo actual y mi puntuación de sostenibilidad. Mi ID debería ser TV001."},
        {"input": "Estoy tratando de hacer un seguimiento de mi impacto ambiental. Creo que mi número de cliente es el mismo que usé para instalar el sistema de conservación de agua y el cargador de vehículos eléctricos. ¿Podrías ayudarme a encontrar mi información?"}
    ],
    outputs=[
        {"tool": "get_client_info", "args": {"client_id": "TV001"}},
        {"tool": "get_client_info", "args": {"client_id": "TV002"}},
        {"tool": "get_client_info", "args": {"client_id": "TV003"}},
        {"tool": "get_client_info", "args": {"client_id": "TV002"}},
        {"tool": "get_client_info", "args": {"client_id": "TV001"}},
        {"tool": "get_client_info", "args": {"client_id": "TV002"}},
        {"tool": "get_client_info", "args": {"client_id": "TV003"}},
        {"tool": "get_client_info", "args": {"client_id": "TV002"}},
        {"tool": "get_client_info", "args": {"client_id": "TV001"}},
        {"tool": "get_client_info", "args": {"client_id": "TV003"}}
    ],
    dataset_id=dataset.id
)

#### App

In [ ]:
#AI App

from agent import Agent
from langchain_chroma import Chroma
from tools import create_retriever_tool_from_vectorstore, create_get_client_info_tool
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
from langchain_openai import OpenAIEmbeddings
persist_directory = "./chroma_db"
vectorstore = Chroma(
    collection_name="rag-chroma",
    embedding_function=OpenAIEmbeddings(),
    persist_directory=persist_directory
)

tools = [create_retriever_tool_from_vectorstore(vectorstore), create_get_client_info_tool()]

from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
        ("system", "Sos un asistente que responde preguntas sobre la empresa TechnoVerde S.A. Para preguntas relacionadas a la empresa, responde utilizando la informacion que tenes disponible sobre la misma, no inventes informacion. Si no conoces la respuesta, simplemente decí que no lo sabes y disculpate por no poder ayudar"),
    ])

In [ ]:
def get_client_info_used(input_dict):
    try:
        agent = Agent(model_type="openai", prompt=template, tools=tools)
        human_message = HumanMessage(content=input_dict['input'])
        state = agent.invoke([human_message])

        if isinstance(state, dict) and "messages" in state and len(state["messages"]) > 0:
            messages = state["messages"]

            # Buscar en los mensajes si hay un ToolMessage con name 'get_client_info'
            for message in messages:
                if isinstance(message, ToolMessage) and message.name == 'get_client_info':
                    # Obtener el ID de la llamada a la herramienta
                    tool_call_id = message.tool_call_id

                    # Buscar en los mensajes anteriores el AIMessage que hizo la llamada a la herramienta
                    for prior_message in messages:
                        if isinstance(prior_message, AIMessage) and hasattr(prior_message, 'tool_calls'):
                            for tool_call in prior_message.tool_calls:
                                if (tool_call['id'] == tool_call_id) and (tool_call['name'] == 'get_client_info'):
                                    return {
                                        "tool": "get_client_info",
                                        "args": tool_call['args']
                                    }

            # Si no se encontró un ToolMessage con name 'get_client_info'
            return {"error": "No se encontró un 'ToolMessage' con name 'get_client_info' en el estado."}
        else:
            return {"error": "No se encontraron mensajes en el estado."}
    except Exception as e:
        return {"error": str(e)}

In [ ]:
get_client_info_used({"input": "Hola, necesito consultar mi información de cuenta. Mi número de cliente es TV001. ¿Me podrías ayudar?"})

#### Evaluator

In [ ]:
# Custom evaluator
def compare_tool_calls(run: Run, example: Example) -> dict:
    expected_dict = example.outputs
    returned_dict = run.outputs
    
    is_match = expected_dict == returned_dict
    
    if is_match:
        return {"key": "vault_map_match", "score": 1, "comment": "Maps match exactly"}
    else:
        mismatches = []
        for key in set(expected_dict.keys()) | set(returned_dict.keys()):
            if key not in expected_dict:
                mismatches.append(f"Unexpected key in returned map: {key}")
            elif key not in returned_dict:
                mismatches.append(f"Missing key in returned map: {key}")
            elif expected_dict[key] != returned_dict[key]:
                mismatches.append(f"Mismatch for key {key}: Expected {expected_dict[key]}, Got {returned_dict[key]}")
        
        return {
            "key": "vault_map_match", 
            "score": 0, 
            "comment": "Maps do not match. Mismatches: " + "; ".join(mismatches)
        }


In [ ]:
dataset_name = "Llamado a la tool de informacion del cliente"
experiment_results = evaluate(
    get_client_info_used,
    data=dataset_name,
    evaluators=[compare_tool_calls],
    experiment_prefix="compare-tool-calls-gpt-4o",

)